# RI Prediction Model

### Prediction of Hurricane Rapid Intensification

Project Summary: Storms that undergo rapid intensification (RI) are associated with the highest forecat errors and larger economic losses.
    To reduce the damage caused by the hurricane, accurate prediction of hurricane intensity is critical. This project aims to improve previous models that predict whether or not hurricane will experience RI within 24 hours.
    
Project Goal: Finding a model that effectively predicts whether the hurricane will or will not experience a Rapid Intensification (RI)

In [461]:
# Python code to build Machine Learning model for hurricane intensity forecast  
import pandas as pd # For data manipulation and analysis
pd.set_option('display.max_columns', 500)
import numpy as np # For scientific computing
import statistics
import time



#Machine Learning Tools
import sklearn # For machine learning library
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import KFold

#from sklearn.ensemble import ExtraTreesClassifier    # Extra tree classifier
from sklearn.metrics import confusion_matrix # Compute confusion matrix to evaluate the accuracy of a classification.
from sklearn.metrics import brier_score_loss  # Compute the Brier score
from sklearn import metrics
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import brier_score_loss  # Compute the Brier score
from sklearn.model_selection import GridSearchCV


import matplotlib.pyplot as plt  #plotting library


#Displaying More Rows
pd.options.display.max_rows = 500

## Importing Data and Data Inspection

Let's take a look closer at each data set. Few of the data characteristics that we are looking for:
- column names
- data type
- number of rows
- missing values (null)
- plausible features that could impact survival rate
- target column

In [462]:
# Set up the location of the data
fname='Dataset_SHIPS_RII_ATL.csv'
#fname='Dataset_SHIPS_RII_EPAC.csv'

# Read SHIPS data
ships = pd.read_csv(fname)
ships

,NAME,DATE,HOUR,VMX0,LAT,LON,MSLP,ID,DELV12,DELV24,DELV36,DELV48,PER,SHRD,D200,RHLO,PX30,SDBT,POT,OHC,TPW,PC2,U200,TPWC,AVBT,RSST
0,ALEX,980727,12,25,11.3,-25.4,1009,AL011998,0,5,10,10,9999,6.3,103,68,72,13.8,-101,12,0,-58,-7.9,55.7,-473,27.4
1,ALEX,980727,18,25,11.7,-27.2,1009,AL011998,0,5,10,10,9999,11.2,118,69,55,12.6,-102,17,0,-10,-6.4,55.7,-360,27.4
2,ALEX,980728,0,25,12.2,-29.2,1009,AL011998,5,10,10,10,0,8.6,116,71,70,12.8,-105,21,0,-3,-8.8,56.9,-381,27.4
3,ALEX,980728,6,25,12.6,-31.3,1008,AL011998,5,10,10,15,0,12.2,91,71,57,12.2,-100,29,0,-44,-6.0,52.7,-481,27.2
4,ALEX,980728,12,30,12.9,-33.3,1007,AL011998,5,5,5,10,5,10.5,88,71,83,10.1,-89,15,190,-46,-6.5,55.3,-516,27.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7400,RINA,171108,0,45,34.6,-48.7,999,AL192017,0,0,9999,9999,5,20.5,29,71,31,24.3,-37,0,1000,123,22.0,42.9,-35,23.5
7401,RINA,171108,6,50,36.4,-48.7,996,AL192017,-5,9999,9999,9999,10,20.8,19,73,15,14.7,-28,0,969,90,21.8,43.0,-121,22.9
7402,RINA,171108,12,45,38.3,-48.8,994,AL192017,0,9999,9999,9999,0,21.2,12,74,9,13.5,-29,0,966,126,23.0,43.0,-38,22.7
7403,RINA,171108,18,45,40.1,-49.0,992,AL192017,9999,9999,9999,9999,-5,22.5,14,70,7,11.5,-14,0,948,75,28.4,44.3,-166,21.3


In [463]:
# Set all 9999s as NaNs
ships = ships.replace(9999,np.NaN)

In [464]:
# Function: Counts all null values in each columns
def count_null(data):
    null_each_col={}

    for col in data.columns:
        null_count=data[col].isnull().sum()
        null_each_col[col]=null_count
    return null_each_col

In [465]:
null_counts = count_null(ships)
null_counts

# There are a lot of null values in DELV12, DELV24, DELV36, DELV48, PER, PX30, SDBT, OHC, PC2, AVBT
# DELV12, DELV24, DELV36, DELV48 will be dropped during the data cleaning since they will not be used as the predictor.


{'NAME': 0,
 'DATE': 0,
 'HOUR': 0,
 'VMX0': 0,
 'LAT': 0,
 'LON': 0,
 'MSLP': 0,
 'ID': 0,
 'DELV12': 718,
 'DELV24': 1427,
 'DELV36': 2082,
 'DELV48': 2682,
 'PER': 1456,
 'SHRD': 0,
 'D200': 0,
 'RHLO': 0,
 'PX30': 402,
 'SDBT': 402,
 'POT': 0,
 'OHC': 19,
 'TPW': 0,
 'PC2': 387,
 'U200': 0,
 'TPWC': 0,
 'AVBT': 402,
 'RSST': 0}

## Parameters Set-up

Setting up RI threshold, train/forecast threshold, target variable/feature, and climatoloogy constant

In [466]:
# Year range for training and validating
year_train=['1998','2008']

# Year range for forecast
year_fcst=['2009','2017']

# Variable name for predictand
TargetName='DELV24'

# Threshold of Rapid Intensification 
RIValue=30

# Climatology of RI (30 kt) frequency at Atlantic basin (Kaplan et al. 2015)
clim=0.125   #ATL 30 kt
#clim=0.084   #EPAC 30 kt

## Data Cleaning

- Cleaning Date Column
- Creating Target Column 'TAR'
- Dropping all DELV Columns
- Dropping all null values

In [467]:
# Pad the date columns with 00 for the year 2000
ships['DATE'] = ships['DATE'].apply(lambda x: str(x).zfill(6))

# Extract month from date
ships['MONTH'] = ships['DATE'].apply(lambda x: str(x)[2:4])

# Extract year from date
ships['YEAR'] = ships['DATE'].apply(lambda x: ('19' + str(x)[0:2]) if (str(x)[0:1]!= '0' and str(x)[0:1]!= '1') else ('20' + str(x)[0:2]))
ships.head()

# Set the target column
ships['TAR'] = ships[TargetName].apply(lambda x: 1 if x >= RIValue else 0)

# Dropping all DELV columns
# - Dropping DELV columns before droping null values saved 1971 data points
ships=ships.drop(['DELV12', 'DELV24', 'DELV36', 'DELV48'], axis=1)

# drop NaNs
ships=ships.dropna()
ships

,NAME,DATE,HOUR,VMX0,LAT,LON,MSLP,ID,PER,SHRD,D200,RHLO,PX30,SDBT,POT,OHC,TPW,PC2,U200,TPWC,AVBT,RSST,MONTH,YEAR,TAR
2,ALEX,980728,0,25,12.2,-29.2,1009,AL011998,0.0,8.6,116,71,70.0,12.8,-105,21.0,0,-3.0,-8.8,56.9,-381.0,27.4,07,1998,0
3,ALEX,980728,6,25,12.6,-31.3,1008,AL011998,0.0,12.2,91,71,57.0,12.2,-100,29.0,0,-44.0,-6.0,52.7,-481.0,27.2,07,1998,0
4,ALEX,980728,12,30,12.9,-33.3,1007,AL011998,5.0,10.5,88,71,83.0,10.1,-89,15.0,190,-46.0,-6.5,55.3,-516.0,27.1,07,1998,0
5,ALEX,980728,18,30,13.1,-35.1,1006,AL011998,5.0,9.7,44,72,35.0,15.9,-86,22.0,15,36.0,-7.2,56.6,-270.0,27.1,07,1998,0
6,ALEX,980729,0,35,13.3,-36.8,1005,AL011998,5.0,9.9,37,74,56.0,14.5,-80,24.0,0,-6.0,-10.3,57.7,-443.0,27.1,07,1998,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7400,RINA,171108,0,45,34.6,-48.7,999,AL192017,5.0,20.5,29,71,31.0,24.3,-37,0.0,1000,123.0,22.0,42.9,-35.0,23.5,11,2017,0
7401,RINA,171108,6,50,36.4,-48.7,996,AL192017,10.0,20.8,19,73,15.0,14.7,-28,0.0,969,90.0,21.8,43.0,-121.0,22.9,11,2017,0
7402,RINA,171108,12,45,38.3,-48.8,994,AL192017,0.0,21.2,12,74,9.0,13.5,-29,0.0,966,126.0,23.0,43.0,-38.0,22.7,11,2017,0
7403,RINA,171108,18,45,40.1,-49.0,992,AL192017,-5.0,22.5,14,70,7.0,11.5,-14,0.0,948,75.0,28.4,44.3,-166.0,21.3,11,2017,0


## Creating Grids for Data Sets, Sampling Methods, and Hyperparameters

- Predictor Sets: Sets with different predictor features
- Data Sets:      Data Sets with different sampling methods
- Model Sets:     Different types of models 

In [468]:
#Predictor Sets

Predictor_Sets=[]

# Adding different features into the list
Predictor_Sets.append(('Set1', ['PER','SHRD','D200','TPW','PC2','SDBT','POT','OHC','VMX0']))
Predictor_Sets.append(('Set2', ['PER','SHRD','D200','TPW','PC2','POT','OHC','VMX0']))
Predictor_Sets.append(('Set3', ['PER','SHRD','D200','TPW','SDBT','POT','OHC','VMX0']))
Predictor_Sets.append(('Kaplan 2015', ['PER','SHRD','D200','RHLO','PX30','SDBT','POT','OHC']))
Predictor_Sets.append(('Kaplan 2015 Imp', ['PER','SHRD','D200','TPW','PC2','SDBT','POT','OHC','VMX0'])) #ICDA didn't exist in the dataset

In [469]:
# This function normalizes the provided data and desired column names

def normalization(data,column_names):
    data_nur=data[column_names]
    data_nur=(data_nur-data_nur.mean())/(data_nur.std()) #standard normal distribution
    data[column_names]=data_nur[column_names]
    return data


In [470]:
# Normalized and Non-Normalized Data Sets

Range_Sets=[]

num_col=['VMX0','PER','SHRD','D200','RHLO','PX30','SDBT','POT','OHC','TPW','PC2','U200','TPWC','AVBT','RSST']
ships_nor=normalization(ships,num_col)




# Adding created data set into a series
Range_Sets.append(('Normalized', ships_nor))
Range_Sets.append(('Not Normalized', ships))

In [471]:
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold

# X, y = make_classification(n_samples=1000, n_classes=2, weights=[0.99, 0.01], flip_y=0, random_state=1)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

# enumerate the splits and summarize the distributions
# for train_ix, test_ix in kfold.split(X, y):
# 	# select rows
# 	train_X, test_X = X[train_ix], X[test_ix]

In [472]:
# df_features = ships[['VMX0','PER','SHRD','D200','RHLO','PX30','SDBT','POT','OHC','TPW','PC2','U200','TPWC','AVBT','RSST']]
# df_target = ships['TAR']
# for train_index, fcst_index in kfold.split(df_features,df_target):
    
#     print(fcst_index)

In [473]:
# Model Sets

# Models = []

# # Adding different machine learning models into the list
# # Models.append(('Logistic_Regression', LogisticRegression(solver='liblinear', multi_class='ovr')))
# Models.append(('K_Neighbors_Classifier', KNeighborsClassifier(n_neighbors=15)))
# Models.append(('Decision_Tree', DecisionTreeClassifier(random_state=1,min_samples_split=10,max_depth=5)))
# Models.append(('Random_Forest', RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=66, max_depth=6, min_samples_leaf=2, class_weight='balanced')))
# Models.append(('Neural_Network_Logistic', MLPClassifier((10,10), activation='logistic',max_iter=3000)))
# Models.append(('Neural_Network_ReLU', MLPClassifier((10,10), activation='relu',max_iter=3000)))

In [474]:
#================================
# Create ensemble model
#================================
import operator
from sklearn.base import BaseEstimator
from sklearn.base import ClassifierMixin

class EnsembleClassifier(BaseEstimator, ClassifierMixin):
    """
    Ensemble classifier for scikit-learn estimators.
    Parameters
    ----------
    clf : `iterable`
      A list of scikit-learn classifier objects.
    weights : `list` (default: `None`)
      If `None`, the majority rule voting will be applied to the predicted class labels.
        If a list of weights (`float` or `int`) is provided, the averaged raw probabilities (via `predict_proba`)
        will be used to determine the most confident class label.
    """
    def __init__(self, clfs, weights=None):
        self.clfs = clfs
        self.weights = weights
    def fit(self, X, y):
        """
        Fit the scikit-learn estimators.
        Parameters
        ----------
        X : numpy array, shape = [n_samples, n_features]
            Training data
        y : list or numpy array, shape = [n_samples]
            Class labels
        """
        for clf in self.clfs:
            clf.fit(X, y)
    def predict(self, X):
        """
        Parameters
        ----------
        X : numpy array, shape = [n_samples, n_features]
        Returns
        ----------
        maj : list or numpy array, shape = [n_samples]
            Predicted class labels by majority rule

        """
        self.classes_ = np.asarray([clf.predict(X) for clf in self.clfs])
        if self.weights:
            avg = self.predict_proba(X)
            maj = np.apply_along_axis(lambda x: max(enumerate(x), key=operator.itemgetter(1))[0], axis=1, arr=avg)
        else:
            maj = np.asarray([np.argmax(np.bincount(self.classes_[:,c])) for c in range(self.classes_.shape[1])])
        return maj
    def predict_proba(self, X):
        """
        Parameters
        ----------
        X : numpy array, shape = [n_samples, n_features]
        Returns
        ----------
        avg : list or numpy array, shape = [n_samples, n_probabilities]
            Weighted average probability for each class per sample.
        """
        self.probas_ = [clf.predict_proba(X) for clf in self.clfs]
        avg = np.average(self.probas_, axis=0, weights=self.weights)
        return avg

In [475]:
# Creating a list of models

models = {
    'Logistic_Regression': LogisticRegression(),
    'K_Neighbors_Classifier': KNeighborsClassifier(),
    'Decision_Tree': DecisionTreeClassifier(),
    'Random_Forest':RandomForestClassifier(),
#     'Neural_Network': MLPClassifier()
}

# Creating Stacking Classifier and Ensemble Using some of the models
estimators1 = [
    ('K_Neighbors_Classifier', KNeighborsClassifier(n_neighbors=18)),
    ('Decision_Tree', DecisionTreeClassifier(random_state=1,min_samples_split=10,max_depth=5)),
    ('Random_Forest', RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=66, max_depth=6, min_samples_leaf=2, class_weight='balanced'))
]

estimators2 = [
    ('K_Neighbors_Classifier', KNeighborsClassifier(n_neighbors=18)),
    ('Decision_Tree', DecisionTreeClassifier(random_state=1,min_samples_split=10,max_depth=5)),
    ('Random_Forest', RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=66, max_depth=6, min_samples_leaf=2, class_weight='balanced')),
    ('Logistic_Regression', LogisticRegression(solver='liblinear', multi_class='ovr'))
]
weights1=[0.25,0.25,0.25,0.25]
weights2=[0.1,0.2,0.2,0.5]

models['Stacking Classifier']=StackingClassifier(estimators=estimators1, final_estimator=LogisticRegression(solver='liblinear', multi_class='ovr'))
models['Ensemble Classifier1']=EnsembleClassifier(clfs=[ estimators2[0][1], estimators2[1][1], estimators2[2][1], estimators2[3][1] ], weights=weights1)
models['Ensemble Classifier2']=EnsembleClassifier(clfs=[ estimators2[0][1], estimators2[1][1], estimators2[2][1], estimators2[3][1] ], weights=weights2)


# Setting parameters for the models
params = {
    'Logistic_Regression': {"C":np.logspace(-3,3,7), "penalty":['11','l2']},
    'K_Neighbors_Classifier': { 'n_neighbors': [10, 18] },
    'Decision_Tree': { 'min_samples_split': [5, 10], 'max_depth':[5,10]},
    'Random_Forest': { 'n_estimators': [16, 32] },
    'Stacking Classifier':{},
    'Ensemble Classifier1':{},
    'Ensemble Classifier2':{}

}
#     'Neural_Network': {'hidden_layer_sizes': [(10,30,10),(20,)],'activation': ['tanh', 'relu'],
#                                 'solver': ['sgd', 'adam','logistic','relu'],'alpha': [0.0001, 0.05],
#                                 'learning_rate': ['constant','adaptive']}



In [476]:


model_keys=models.keys()

for key in model_keys:
    model=models[key]
    param=params[key]
    

In [477]:
# Creating different sampling method lists

In [478]:
def nonsampling(df_train_feature,df_train_target,feature):
    return df_train_feature, df_train_target

In [479]:
def undersampling(df_train_feature,df_train_target,feature):
    df_train=df_train_feature
    df_train['TAR']=df_train_target
    TAR_0 = df_train[df_train['TAR']==0]
    TAR_1 = df_train[df_train['TAR']==1]
    TAR_0_count_train,TAR_1_count_train=df_train['TAR'].value_counts()

    TAR_0_under=TAR_0.sample(TAR_1_count_train)
    df_train = pd.concat([TAR_0_under,TAR_1], axis=0)
    
    df_train_feature = df_train[feature]
    df_train_target = df_train['TAR']
    
    return df_train_feature, df_train_target

In [480]:
def oversampling(df_train_feature,df_train_target,feature):
    df_train=df_train_feature
    df_train['TAR']=df_train_target
    TAR_0 = df_train[df_train['TAR']==0]
    TAR_1 = df_train[df_train['TAR']==1]
    TAR_0_count_train,TAR_1_count_train=df_train['TAR'].value_counts()
    
    TAR_1_over=TAR_1.sample(TAR_0_count_train,replace=True)
    df_train = pd.concat([TAR_0,TAR_1_over], axis=0)
    
    df_train_feature = df_train[feature]
    df_train_target = df_train['TAR']
    
    return df_train_feature, df_train_target

In [481]:

sampling_methods = {
    'Non_Sampling': nonsampling,
    'Under_Sampling': undersampling,
    'Over_Sampling': oversampling
}

## Machine Learning Calculator

ML_Calc function takes in predictor,data, and model sets to predict the given target variable of the forecast data

In [482]:
def ML_Calc(predictors,range_sets,models,params,target):
    start_time = time.time()

    results=pd.DataFrame()
    grid_searches = {}

#     data_output=pd.DataFrame(index=ships.index)
#     i=1
    for name_pred,features in predictors:
        print(name_pred)
        for name_range,df_ships in range_sets:            
            df_features = df_ships[features]
            df_target = df_ships['TAR']
            
            for method_key in sampling_methods:
                
#                 prediction_list=[]
                
                for key in model_keys:
                
                    false_neg_list=[]
                    false_pos_list=[]
                    true_pos_list=[]
                    true_neg_list=[]
                    
#                     data_model = pd.DataFrame(index=ships.index)

                    
                    
                    for train_index, fcst_index in kfold.split(df_features,df_target):
                        
                        df_train_feature, df_train_target = df_features.iloc[train_index], df_target.iloc[train_index]
                        df_fcst_feature, df_fcst_target = df_features.iloc[fcst_index], df_target.iloc[fcst_index]
                        
                        
                        method=sampling_methods[method_key]
                        df_train_feature,df_train_target= method(df_train_feature,df_train_target,features)
                        
                        print("Running GridSearchCV for %s." % key)

                        model=models[key]
                        param=params[key]
                        if model == 'Stacking Classifier' or model == 'Ensemble Classifier':
                            model.fit(df_train_feature,df_train_target)
                            prediction_fcst = model.predict(df_fcst_feature)

                        else:
                            gs = GridSearchCV(model, param, scoring='recall')
                            gs.fit(df_train_feature,df_train_target)
                            grid_searches[key] = gs
                            prediction_fcst = gs.predict(df_fcst_feature)
                        cmatrix_fcst = confusion_matrix(df_fcst_target, prediction_fcst)
                        
                        
                        
#                         data_one_model=pd.DataFrame(index=fcst_index)
#                         data_one_model[i]=prediction_fcst
#                         data_output = data_output.merge(data_one_model, how='outer', left_index=True, right_index=True)
                        
                        
                        false_neg_list.append(cmatrix_fcst[1,0])
                        false_pos_list.append(cmatrix_fcst[0,1])
                        true_pos_list.append(cmatrix_fcst[1,1])
                        true_neg_list.append(cmatrix_fcst[0,0])
                    
#                         prediction_list.append(prediction_fcst)
                        
#                         i+=1
                    
                    
#                     data_output = data_model.merge(data_model, how='outer', left_index=True, right_index=True)
                    
                    
                    
                    false_neg=statistics.mean(false_neg_list)
                    false_pos=statistics.mean(false_pos_list)
                    true_pos=statistics.mean(true_pos_list)
                    true_neg=statistics.mean(true_neg_list)

                    pss=((cmatrix_fcst[0,0] * cmatrix_fcst[1,1]) - (cmatrix_fcst[0,1] * cmatrix_fcst[1,0])) * 1.0 / ((cmatrix_fcst[1,1] + cmatrix_fcst[1,0]) * (cmatrix_fcst[0,1] + cmatrix_fcst[0,0]))
                    far=(cmatrix_fcst[0,1] * 1.0) / (cmatrix_fcst[0,1] + cmatrix_fcst[1,1])
                    pod=(cmatrix_fcst[1,1] * 1.0) / (cmatrix_fcst[1,0] + cmatrix_fcst[1,1]) 
                    precision=precision_score(df_fcst_target, prediction_fcst, average=None)[1]
                    recall=recall_score(df_fcst_target, prediction_fcst)
                    f1=f1_score(df_fcst_target, prediction_fcst)
                    brier_score=brier_score_loss(df_fcst_target,prediction_fcst)

                    results=results.append({"Predictor Set":name_pred,"Normalized?":name_range, 
                                            "ML Name":key,"True Positive":true_pos,"True Negative":true_neg,"False Negative":false_neg,
                                            "False Positive":false_pos,"Sampling Methods":method_key,
                                            "PSS":pss,"FAR":far,"POD":pod, "Recall":recall,"Precision":precision,"F1":f1, 
                                            "Brier Score":brier_score}, 
                                           ignore_index = True)




    results=results[["Normalized?","Predictor Set","ML Name","Sampling Methods","True Positive","True Negative","False Negative",
                     "False Positive","PSS","FAR","POD","F1","Precision","Brier Score"]]
    results=results.sort_values('False Negative')
    end_time = time.time()
    elapsed_time=end_time-start_time
    print(elapsed_time)
    return results


In [483]:
Result=ML_Calc(Predictor_Sets,Range_Sets,models,params,'TAR')

Set1
Running GridSearchCV for Logistic_Regression.


C:\Users\kwonk\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\kwonk\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\kwonk\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\kwonk\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 439, in _check_solver
    " got %s." % (all_penalties, penalty))
ValueError: Logistic Regression supports only penalties in ['l1', 'l2', 'elasticnet', 'none'], got 11.

  FitFailedWarning)
C:\Users\kwonk\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. 

Running GridSearchCV for Logistic_Regression.


C:\Users\kwonk\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\kwonk\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\kwonk\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\kwonk\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 439, in _check_solver
    " got %s." % (all_penalties, penalty))
ValueError: Logistic Regression supports only penalties in ['l1', 'l2', 'elasticnet', 'none'], got 11.

  FitFailedWarning)
C:\Users\kwonk\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. 

Running GridSearchCV for Logistic_Regression.


...

Running GridSearchCV for Ensemble Classifier2.


C:\Users\kwonk\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Running GridSearchCV for Ensemble Classifier2.


C:\Users\kwonk\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Running GridSearchCV for Ensemble Classifier2.
3408.740987062454


In [484]:
Result.sort_values('PSS',ascending=False)

,Normalized?,Predictor Set,ML Name,Sampling Methods,True Positive,True Negative,False Negative,False Positive,PSS,FAR,POD,F1,Precision,Brier Score
145,Normalized,Kaplan 2015,Ensemble Classifier1,Over_Sampling,55.0,846.0,11.0,229.0,0.581784,0.816949,0.805970,0.298343,0.183051,0.222417
83,Not Normalized,Set2,Ensemble Classifier2,Over_Sampling,56.0,826.0,10.0,249.0,0.581784,0.816949,0.805970,0.298343,0.183051,0.222417
35,Not Normalized,Set1,Logistic_Regression,Over_Sampling,56.0,798.0,10.0,277.0,0.580937,0.830303,0.835821,0.282116,0.169697,0.249562
188,Normalized,Kaplan 2015 Imp,Ensemble Classifier2,Over_Sampling,57.0,831.0,10.0,243.0,0.576203,0.820598,0.805970,0.293478,0.179402,0.227671
20,Normalized,Set1,Ensemble Classifier2,Over_Sampling,57.0,830.0,10.0,245.0,0.571552,0.823529,0.805970,0.289544,0.176471,0.232049
208,Not Normalized,Kaplan 2015 Imp,Ensemble Classifier1,Over_Sampling,56.0,842.0,11.0,233.0,0.569649,0.817869,0.791045,0.296089,0.182131,0.220665
166,Not Normalized,Kaplan 2015,Ensemble Classifier1,Over_Sampling,57.0,841.0,10.0,234.0,0.563138,0.822148,0.791045,0.290411,0.177852,0.226795
41,Not Normalized,Set1,Ensemble Classifier2,Over_Sampling,56.0,835.0,11.0,240.0,0.563138,0.822148,0.791045,0.290411,0.177852,0.226795
40,Not Normalized,Set1,Ensemble Classifier1,Over_Sampling,57.0,832.0,9.0,242.0,0.562208,0.822742,0.791045,0.289617,0.177258,0.227671
203,Not Normalized,Kaplan 2015 Imp,Logistic_Regression,Over_Sampling,56.0,799.0,11.0,276.0,0.561361,0.835329,0.820896,0.274314,0.164671,0.254816
